In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Add the necessary libraries

In [ ]:
import pandas as pd # thư viện làm việc với bảng
import numpy as np # thư viện tính toán trên ma trận số 
from collections import defaultdict # dictionary có giá trị default
from collections import Counter # hàm đếm tạo thành dictionary
import matplotlib.pyplot as plt # thư viện vẽ biểu đồ matplotlib
plt.style.use('ggplot') # chỉnh theme cho biểu đồ
import seaborn as sns # thư viện vẽ biểu đồ, đẹp hơn matplotlib

In [ ]:
figsize = (15, 7)

# Import data

In [ ]:
df = pd.read_csv('/kaggle/input/pokemon-dataset-with-team-combat/pokemon.csv').drop('#', axis=1)
df.head()

#### I created the `Total` columns is the sum of the rest stats of pokemon

In [ ]:
df['Total'] = df['HP'] + df['Attack'] + df['Defense'] + df['Sp. Atk'] + df['Sp. Def'] + df['Speed']

# Basic info of dataset

In [ ]:
df.info()

#### Remove the Pokemon that no name

In [ ]:
df = df[~df.Name.isna()]

#### Fill `Type 2` with empty character

In [ ]:
df['Type 2'].fillna('', inplace=True)

In [ ]:
df.info()

In [ ]:
df[df['Name'].str.contains('Mega')]

In [ ]:
df['Name'] = df['Name'].str.replace('.*(?=Mega)', '')

In [ ]:
df[df['Name'].str.contains('Mega')]

# Centroid values
### Mode, Median and Mean

![mmm](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/mmm.png)

In [ ]:
df.describe()

# Boxplot
![boxplot](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/boxplot.png)

### Observe the boxplot of the attributes of the Pokemon

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
sns.boxplot(data=df.iloc[:, 3:-1].melt(), x='variable', y='value')
plt.tight_layout()
plt.show()

#### Question:
- Are all outliers `Legendary` pokemon?

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
sns.boxplot(data=df.iloc[:, 3:].melt(id_vars='Legendary'), x='variable', y='value', hue='Legendary')
plt.tight_layout()
plt.show()

# Histogram

![histogram](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/hist.jpg)

#### Question:
- How `HP` distribution?

In [ ]:
def plot(data, x, is_hist=True, **kwargs):
    plt.figure(figsize=figsize)
    if is_hist:
        sns.histplot(data=data, x=x, **kwargs)
        plt.axvline(data[x].mean(), ls='-.', c='blue', lw=4, label='mean')
        plt.axvline(data[x].median(), ls='-.', c='red', lw=4, label='median')
    else:
        sns.boxplot(data=data, x=x, **kwargs)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

In [ ]:
plot(df, 'HP', color='purple')

- Distribution of `Attack`

In [ ]:
plot(df, 'Attack')

# Bar chart and Histogram

#### Question:
- Why histogram instead of barchart?

![bar vs hist](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/hist_vs_bar.png)

# Bar chart

#### Question:
- What is the number of each type of pokemon?

In [ ]:
data = df.groupby('Type 1')['Name'].count().reset_index().sort_values(by='Name').reset_index(drop=True)
display(data.T)

fig, ax = plt.subplots(figsize=figsize)
g = sns.barplot(data=data, x='Type 1', y='Name', ax=ax)
for p in g.patches:
    ax.annotate(
        format(p.get_height(), '.0f'),
        (p.get_x() + p.get_width() / 2, p.get_height()),
        fontsize=12, va='bottom', ha='center', fontweight='bold'
    )
plt.tight_layout()
plt.show()

In [ ]:
data = df.groupby('Type 2')['Name'].count().reset_index().sort_values(by='Name').reset_index(drop=True)
display(data.T)
fig, ax = plt.subplots(figsize=figsize)
g = sns.barplot(data=data, x='Type 2', y='Name', ax=ax)
for p in g.patches:
    ax.annotate(
        format(p.get_height(), '.0f'),
        (p.get_x() + p.get_width() / 2, p.get_height()),
        fontsize=12, va='bottom', ha='center', fontweight='bold'
    )
plt.tight_layout()
plt.show()

# Pie chart

![pie](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/pie.jpg)

### Using pie chart inappropriately

In [ ]:
pie_shape = (15, 6)

#### Question:
- What is the ratio of the systems in Pokemon?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=pie_shape)
df['Type 1'].value_counts().plot(kind='pie', autopct='%.2f%%', fontsize=13, shadow=True, ax=ax[0])
df['Type 2'].value_counts().plot(kind='pie', autopct='%.2f%%', fontsize=13, shadow=True, ax=ax[1])
plt.tight_layout()
plt.show()

## $\rightarrow$ Improper use of pie chart.

### Fix: Use bar chart or reduce the number of properties.

## Use the right pie chart

**$\rightarrow$ Few attributes**

#### Question:
- Number of pokemon of generation?

In [ ]:
plt.figure(figsize=pie_shape)
df['Generation'].value_counts().plot(kind='pie', autopct='%.2f%%', fontsize=15, shadow=True, explode=[0.1, 0, 0, 0, 0, 0])
plt.tight_layout()
plt.show()

# Scatter

![scatter](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/scatter.gif)

#### Question:
- Does `Attack` and `Defense` has correlation?

In [ ]:
plt.figure(figsize=figsize)
sns.scatterplot(data=df, x='Attack', y='Defense')
plt.tight_layout()
plt.show()

# Correlation - correlation between two features

![corr_gif](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/pearson.gif)

### $Correlation \in [-1, 1]$

![corr](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/corr_dir.png)

In [ ]:
plt.figure(figsize=figsize)
sns.regplot(data=df, x='Attack', y='Defense')
plt.tight_layout()
plt.show()

In [ ]:
x = df['Attack']
y = df['Defense']
x.corr(y)

![corr](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/corr.png)

# Heatmap

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(data=df.corr(), annot=True, fmt='.2f', vmin=-1, vmax=1)
plt.tight_layout()
plt.show()

# Lineplot

#### Question:
- How many Pokemon per generation?

In [ ]:
data = df['Generation'].value_counts().reset_index().rename(columns={'Generation': 'count', 'index': 'Generation'})
data.sort_values(by='Generation', inplace=True)
data

In [ ]:
plt.figure(figsize=figsize)
sns.lineplot(data=data, x='Generation', y='count', lw=3)
sns.scatterplot(data=data, x='Generation', y='count', s=300, color='orange')
plt.show()

# Barchart Race

In [ ]:
from IPython.display import HTML
HTML('<iframe width="1000" height="500" src="https://www.youtube.com/embed/HY9D42M_0d8" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

# Explore Data Analysis

#### Question:
- How to see the number of Pokemon with 1 or 2 types like?

In [ ]:
corr_dict = defaultdict(dict)
for t1 in set(df['Type 1'].unique()):
    for t2 in set(df['Type 2'].unique()):
        corr_dict[t1][t2] = df[(df['Type 1'] == t1) & (df['Type 2'] == t2)]['Name'].count()
data = pd.DataFrame(corr_dict)

mask = np.zeros_like(data)
mask[data == 0] = True

In [ ]:
data

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(data=data, annot=True, fmt='d', mask=mask, cmap='Purples', cbar=False)
plt.xlabel('Type 1', fontsize=12)
plt.ylabel('Type 2', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
hist_figsize = (15, 2)
binwidth = 5
f, ax = plt.subplots(1, 3, sharey=False, figsize=hist_figsize)
sns.histplot(data=df['HP'], ax=ax[0], binwidth=binwidth)
sns.histplot(data=df['Attack'], ax=ax[1], binwidth=binwidth)
sns.histplot(data=df['Defense'], ax=ax[2], binwidth=binwidth)
for i in range(3):
    ax[i].grid(False)
f.tight_layout()
    
f, ax = plt.subplots(1, 3, sharey=False, figsize=hist_figsize)
sns.histplot(data=df['Sp. Atk'], ax=ax[0], binwidth=binwidth)
sns.histplot(data=df['Sp. Def'], ax=ax[1], binwidth=binwidth)
sns.histplot(data=df['Speed'], ax=ax[2], binwidth=binwidth)
for i in range(3):
    ax[i].grid(False)
f.tight_layout()
    
f, ax = plt.subplots(figsize=hist_figsize)
sns.histplot(data=df['Total'], ax=ax, binwidth=binwidth)
ax.grid(False)
f.tight_layout()
plt.show()

In [ ]:
hist_figsize = (15, 2)
binwidth = 5

f, ax = plt.subplots(1, 3, sharey=False, figsize=hist_figsize)
sns.kdeplot(df['HP'], ax=ax[0], fill=True)
sns.kdeplot(data=df['Attack'], ax=ax[1], fill=True)
sns.kdeplot(data=df['Defense'], ax=ax[2], fill=True)
f.tight_layout()
for i in range(3):
    ax[i].grid(False)
    
f, ax = plt.subplots(1, 3, sharey=False, figsize=hist_figsize)
sns.kdeplot(data=df['Sp. Atk'], ax=ax[0], fill=True)
sns.kdeplot(data=df['Sp. Def'], ax=ax[1], fill=True)
sns.kdeplot(data=df['Speed'], ax=ax[2], fill=True)
f.tight_layout()

for i in range(3):
    ax[i].grid(False)
f, ax = plt.subplots(figsize=hist_figsize)
sns.kdeplot(data=df['Total'], ax=ax, fill=True)
ax.grid(False)
f.tight_layout()

plt.show()

In [ ]:
hist_figsize = (15, 2)
binwidth = 5
f, ax = plt.subplots(1, 3, sharey=True, figsize=hist_figsize)
sns.boxplot(data=df, ax=ax[0], x='HP')
sns.boxplot(data=df, ax=ax[1], x='Attack')
sns.boxplot(data=df, ax=ax[2], x='Defense')
f.tight_layout()

f, ax = plt.subplots(1, 3, sharey=True, figsize=hist_figsize)
sns.boxplot(data=df, ax=ax[0], x='Sp. Atk')
sns.boxplot(data=df, ax=ax[1], x='Sp. Def')
sns.boxplot(data=df, ax=ax[2], x='Speed')
f.tight_layout()

f, ax = plt.subplots(figsize=hist_figsize)
sns.boxplot(data=df, ax=ax, x='Total')
f.tight_layout()

plt.show()

In [ ]:
boxplot_figsize = (20, 6)
fig, ax = plt.subplots(1, 2, figsize=boxplot_figsize, sharey=True)
sns.boxplot(data=df.melt(id_vars=['Type 1'], value_vars=['HP']), y='Type 1', x='value', ax=ax[0])
sns.boxplot(data=df.melt(id_vars=['Type 1'], value_vars=['Attack']), y='Type 1', x='value', ax=ax[1])

fig, ax = plt.subplots(1, 2, figsize=boxplot_figsize, sharey=True)
sns.boxplot(data=df.melt(id_vars=['Type 1'], value_vars=['Defense']), y='Type 1', x='value', ax=ax[0])
sns.boxplot(data=df.melt(id_vars=['Type 1'], value_vars=['Sp. Atk']), y='Type 1', x='value', ax=ax[1])
fig.tight_layout()

fig, ax = plt.subplots(1, 2, figsize=boxplot_figsize, sharey=True)
sns.boxplot(data=df.melt(id_vars=['Type 1'], value_vars=['Sp. Def']), y='Type 1', x='value', ax=ax[0])
sns.boxplot(data=df.melt(id_vars=['Type 1'], value_vars=['Speed']), y='Type 1', x='value', ax=ax[1])
fig.tight_layout()

fig, ax = plt.subplots(figsize=boxplot_figsize)
sns.boxplot(data=df.melt(id_vars=['Type 1'], value_vars=['Total']), y='Type 1', x='value', ax=ax)
fig.tight_layout()

plt.show()

In [ ]:
def plot_bar(df, name, title):
    data = df.sort_values(by=name, ascending=False).head(10)
    fig, ax = plt.subplots(figsize=(6, 6))
    g = sns.barplot(data=data, y='Name', x=name)
    for p in g.patches:
        ax.annotate(
            format(p.get_width(), '.0f'),
            (p.get_height() + p.get_width(), p.get_y()),
            ha='left', va='top'
        )
    plt.title(title)
    plt.show()
    return list(data['Name'].values)

# Top 10 Pokemon of stats
### Total, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed

In [ ]:
top_of_tops = []

In [ ]:
top_of_tops += plot_bar(df, name='Total', title='Top 10 Pokemon with the highest total stats')

In [ ]:
top_of_tops += plot_bar(df, name='HP', title='Top 10 Pokemon with the highest health')

In [ ]:
top_of_tops += plot_bar(df, name='Attack', title='Top 10 Pokemon with the highest base attack power')

In [ ]:
top_of_tops += plot_bar(df, name='Defense', title='Top 10 Pokemon with the highest basic defense')

In [ ]:
top_of_tops += plot_bar(df, name='Sp. Atk', title='Top 10 Pokemon with the highest special attack power')

In [ ]:
top_of_tops += plot_bar(df, name='Sp. Def', title='Top 10 Pokemon with the highest special defense')

In [ ]:
top_of_tops += plot_bar(df, name='Speed', title='Top 10 Pokemon with the highest speed')

In [ ]:
top_of_top_df = pd.DataFrame(Counter(top_of_tops).items(), columns=['Name', 'Count']).sort_values(by='Count', ascending=False)
top_of_top_df.head(10)

In [ ]:
plot_bar(top_of_top_df.head(10), name='Count', title='Pokemon at the top of the top 10 charts')

In [ ]:
def plot_radar(name, fig, axis=111):
    index = df[df['Name'] == name].index[0]
    labels=np.array(['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'])
    stats=df.loc[index, labels].values

    angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
    # close the plot
    stats=np.concatenate((stats,[stats[0]]))
    angles=np.concatenate((angles,[angles[0]]))
    labels = np.concatenate((labels, [labels[0]]))
    
    ax = fig.add_subplot(axis, polar=True)
    ax.plot(angles, stats, 'o-', linewidth=2)
    ax.fill(angles, stats, alpha=0.25)
    ax.set_thetagrids(angles * 180/np.pi, labels, fontsize=12, fontweight='bold')
    ax.set_title(name)
    ax.grid(True)
    
    return fig

In [ ]:
fig = plt.figure(figsize=(16, 6))
fig = plot_radar('Mega Mewtwo Y', fig, axis=121)
fig = plot_radar('Mega Mewtwo X', fig, axis=122)
fig.tight_layout()
plt.show()

### We have the story:

You are assigned a task to help the boy Satoshi, a 10-year-old boy, on his way to becoming a professional Pokemon trainer.

![satoshi](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/satoshi.png)

And the professor allowed the boy to choose one of the 3 pokemon below, respectively Bulbasaur, Charmander, Squirtle

![3start](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/3start.jpg)

Satoshi is wondering about one of these 3 Pokemon, looking for advice from you, a professional data scientist. With the knowledge of data exploration so far, if it were you, which Pokemon would you choose from the 3 Pokemon above so that Satoshi could firmly step on the way to becoming a professional Pokemon Trainer.

# Bulbasaur

![saur](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/evolution_saur.png)

In [ ]:
def plot_series_radar(names):
    labels=np.array(['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'])
    angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
    angles=np.concatenate((angles,[angles[0]]))
    labels = np.concatenate((labels, [labels[0]]))

    indexes = []
    stats = []
    for name in names:
        indexes.append(df[df['Name'] == name].index[0])
        stats.append(list(df.loc[indexes[-1], labels].values))

    stats = [stat + stat[0] for stat in stats]

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 12), subplot_kw=dict(polar=True))

    for stat, name in zip(stats, names):
        ax.plot(angles, stat, 'o-', linewidth=2, label=name)
        ax.fill(angles, stat, alpha=0.25)

    ax.set_thetagrids(angles * 180/np.pi, labels, fontsize=12, fontweight='bold')
    # ax.set_title(name)
    ax.grid(True)
    plt.legend(loc='center')
    ax.set_title('Radar chart of %s' % (', '.join(names)))
    plt.show()

In [ ]:
bulbasaur_names = ['Bulbasaur', 'Ivysaur', 'Venusaur', 'Mega Venusaur']
plot_series_radar(bulbasaur_names)

# Charmander

![chamander](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/chamander.jpg)

In [ ]:
charmander_names = ['Charmander', 'Charmeleon', 'Charizard', 'Mega Charizard X', 'Mega Charizard Y']
plot_series_radar(charmander_names)

# Squirtle
![squirtle](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/squirtle.jpg)

In [ ]:
squirtle_names = ['Squirtle', 'Wartortle', 'Blastoise', 'Mega Blastoise']
plot_series_radar(squirtle_names)

## With the last and final analysis, Satoshi chose...

![satoshi_pikachu](https://raw.githubusercontent.com/tuanio/pbl_ds_2021/main/week%203/assets/satoshi_pikachu.png)